# Four queens

In [1]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import XGate
from qiskit_aer import AerSimulator, Aer
from qiskit.visualization import plot_state_city, plot_histogram

In [25]:
row_circuit = QuantumCircuit(4, name="row")

row_circuit.h(0)
row_circuit.h(1)

row_circuit.ccx(0, 1, 2)
row_circuit.x(0)
row_circuit.x(1)
row_circuit.ccx(0, 1, 3)

row_circuit.cx(3, 1)
row_circuit.cx(3, 0)
# row = row_circuit.to_instruction()
row_circuit.measure_all()
row_circuit.draw()

┌───┐     ┌───┐          ┌───┐ ░ ┌─┐         
   q_0: ┤ H ├──■──┤ X ├──■───────┤ X ├─░─┤M├─────────
        ├───┤  │  ├───┤  │  ┌───┐└─┬─┘ ░ └╥┘┌─┐      
   q_1: ┤ H ├──■──┤ X ├──■──┤ X ├──┼───░──╫─┤M├──────
        └───┘┌─┴─┐└───┘  │  └─┬─┘  │   ░  ║ └╥┘┌─┐   
   q_2: ─────┤ X ├───────┼────┼────┼───░──╫──╫─┤M├───
             └───┘     ┌─┴─┐  │    │   ░  ║  ║ └╥┘┌─┐
   q_3: ───────────────┤ X ├──■────■───░──╫──╫──╫─┤M├
                       └───┘           ░  ║  ║  ║ └╥┘
meas: 4/══════════════════════════════════╩══╩══╩══╩═
                                          0  1  2  3

In [ ]:
# Simulator
simulator = AerSimulator(method="statevector", device="GPU")

row_circuit.save_statevector()

# Execution on simulator
row_result = simulator.run(row_circuit).result()

row_sv = row_result.get_statevector(row_circuit)
row_result.get_counts(row_circuit)

In [3]:
qc = QuantumCircuit(25)
n = 4
# 4x4 16 qubits representing 16 position in the board
# built with row gate(rgate), so each row has only one qubit excited (repesenting queen)
for i in range(0, n*n, n):
    qc.append(row, [i for i in range(i, i+n)])

qc = qc.decompose()


In [4]:
# n-1(here, 3) ancillas for column check
qc.h(16)
qc.h(17)
qc.h(18)

for i in range(0, 16, 4):
    qc.cz(16, i)
    qc.cz(17, i+1)
    qc.cz(18, i+2)

qc.h(16)
qc.h(17)
qc.h(18)

In [5]:
# n(n-1)/2 (here, 6) ancillas for diagonal check
qc.x(19)
qc.x(20)
qc.x(21)
qc.x(22)
qc.x(23)
qc.x(24)
# row1, row2 -> qubit19
qc.ccx(0, 5, 19)
qc.ccx(1, 6, 19)
qc.ccx(2, 7, 19)
qc.ccx(1, 4, 19)
qc.ccx(2, 5, 19)
qc.ccx(3, 6, 19)

# row1, row3 -> qubit20
qc.ccx(0, 10, 20)
qc.ccx(1, 11, 20)
qc.ccx(2, 8, 20)
qc.ccx(3, 9, 20)

# row1, row4 -> qubit21
qc.ccx(0, 15, 21)
qc.ccx(3, 12, 21)

# row2, row3 -> qubit22
qc.ccx(4, 9, 22)
qc.ccx(5, 10, 22)
qc.ccx(6, 11, 22)
qc.ccx(5, 8, 22)
qc.ccx(6, 9, 22)
qc.ccx(7, 10, 22)

#row2, row4 -> qubit23
qc.ccx(4, 14, 23)
qc.ccx(5, 15, 23)
qc.ccx(6, 12, 23)
qc.ccx(7, 13, 23)

# row3, row4 -> qubit24
qc.ccx(8, 13, 24)
qc.ccx(9, 14, 24)
qc.ccx(10, 15, 24)
qc.ccx(9, 12, 24)
qc.ccx(10, 13, 24)
qc.ccx(11, 14, 24)

qc.measure_all()

In [ ]:
qc.draw(output="mpl")

In [6]:
# Simulator
simulator = AerSimulator(method="statevector", device="GPU")
# simulator = Aer.get_backend('statevector_simulator')

qc.save_statevector()

# Execution on simulator
result = simulator.run(qc, shots=1024).result()

sv = result.get_statevector(qc)

counts = result.get_counts(qc)
print(len(counts))


253


In [11]:
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler import PassManager, InstructionDurations
from qiskit_ibm_runtime.transpiler.passes.scheduling import (
    ASAPScheduleAnalysis,
    PadDynamicalDecoupling,
)
 
# If you have not previously saved your credentials, follow instructions at
# https://docs.quantum.ibm.com/guides/setup-channel#iqp
# to authenticate with your API token.
service = QiskitRuntimeService(channel="ibm_quantum", token="b8e4542f8f1728ac3463a50ecc8a3739a663a9f9ac08b86a6af9ea4f57af9ad1e21bc8a7a5dae2d0a2134aa414c8ab736ab59702d4040c3ded66f3d5ae1b55f6")

In [20]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from numpy import pi

qreg_q = QuantumRegister(4, 'q')
creg_c = ClassicalRegister(4, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.x(qreg_q[0])
circuit.x(qreg_q[2])
circuit.x(qreg_q[1])
circuit.ccx(qreg_q[0], qreg_q[1], qreg_q[3])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.ccx(qreg_q[1], qreg_q[2], qreg_q[3])
circuit.cx(qreg_q[1], qreg_q[2])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.measure_all()

In [ ]:
backend = service.least_busy(simulator=False, operational=True)

# Convert to an ISA circuit and layout-mapped observables.
# pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
# isa_circuit = pm.run(row_circuit)
# Get gate durations so the transpiler knows how long each operation takes
mc = QuantumCircuit(10)
mc.measure_all()
pm = generate_preset_pass_manager(3, backend=backend)
t_rc = pm.run(mc)


durations = InstructionDurations().from_backend(backend)

# This is the sequence we'll apply to idling qubits
dd_sequence = [XGate(), XGate()]
pm = PassManager([ASAPScheduleAnalysis(durations), PadDynamicalDecoupling(durations, dd_sequence)])
isa_circuit = pm.run(t_rc)
# isa_circuit.draw("mpl", idle_wires=False)

In [ ]:
sampler = Sampler(backend)
job = sampler.run([isa_circuit], shots=16000)

In [ ]:
job_result = job.result()

# This is the result from our single pub, which had six observables,
# so contains information on all six.
counts = job.result()[0].data.meas.get_counts()

In [ ]:
# print(backend)
expected = ['00000001', '00000010', '00000100', '00001000', '00010000', '00100000', '01000000', '10000000']
plot_histogram(dict((k,v) for k,v in counts.items() if k in expected))

In [ ]:
plot_histogram(counts)

In [9]:
# pretty print
def position(key):
    for i in range(0, 16):
        print('q' if key[24-i] == '1' else '.', end=' ')
        if (i+1)%4 == 0:
            print()

In [10]:
# filter out the output, all ancillary bits should be 1
import re
regex = re.compile(r'1{9}[01]+')

for key, value in counts.items():
    if regex.match(key):
        position(key)
        print("count:",value, '\n')

. q . . 
. . . q 
q . . . 
. . q . 
count: 6 

. . q . 
q . . . 
. . . q 
. q . . 
count: 2 

